In [1]:
import gemmi
import pymolviz as pmv

In [2]:
import numpy as np
from xbpy import rdutil

In [3]:
prot = rdutil.read_molecules("4de3.pdb")[0]

In [4]:
positions = []
for atom in prot.GetAtoms():
    positions.append(rdutil.position(atom))
positions = np.array(positions)
min_pos = np.min(positions, axis=0)
max_pos = np.max(positions, axis=0)

In [5]:
buffer = 5
min_pos -= buffer
max_pos += buffer

In [6]:
mtz = gemmi.read_mtz_file("4de3_phases.mtz")
#mtz.switch_to_original_hkl()

In [7]:
mtz.column_labels()

['H',
 'K',
 'L',
 'FREE',
 'FP',
 'SIGFP',
 'FC',
 'PHIC',
 'FC_ALL',
 'PHIC_ALL',
 'FWT',
 'PHWT',
 'DELFWT',
 'PHDELWT',
 'FOM',
 'FC_ALL_LS',
 'PHIC_ALL_LS']

In [8]:
map = mtz.transform_f_phi_to_map("FP", "PHIC", sample_rate = 2.6)

In [9]:
total_std = np.std(map.array, axis=None)

In [10]:
counts = 1/np.array(map.array.shape,dtype=float)
m = gemmi.Mat33()
m.fromlist([[float(counts[0]),0.,0.],[0.,float(counts[1]),0.],[0.,0.,float(counts[2])]])
m = map.unit_cell.fractionalization_matrix.inverse().multiply(m)

In [11]:
m

<gemmi.Mat33 [0.501933, 0, -0.0950668]
             [0, 0.53594, 0]
             [0, 0, 0.518998]>

In [12]:
transform = gemmi.Transform(m, gemmi.Vec3(0,0,0))
values = np.zeros((30,30,30), dtype = np.float32)
map.interpolate_values(values, transform)

In [13]:
indices = np.indices(values.shape).reshape(3,-1).T

In [14]:
for index in indices:
    if not np.allclose(values[tuple(index)], map.get_value(*index)):
        print(index, values[tuple(index)], map.get_value(*index))

In [15]:
positions = (np.array(m.tolist()).reshape(3,3) @ np.indices(values.shape).reshape(3,-1)).T

In [16]:
positions[1]

array([-0.09506676,  0.        ,  0.51899831])

In [17]:
for index in np.indices(values.shape).reshape(3,-1).T[10:]:
    position = np.array(m.tolist()).reshape(3,3) @ index
    target_position = np.array(map.get_position(*index).tolist())
    if not np.allclose(target_position, position):
        print(index, position, target_position)
        break

In [18]:
pmv.Points(positions[values.flatten() > total_std], color = values.flatten()[values.flatten() > total_std]).to_script(name = "reduced_points").write("mtz_test3_points.py")

In [19]:
regular_data = pmv.RegularData(positions, values.flatten(), step_count= np.array(values.shape))

In [20]:

std = regular_data.values.std()
mean = regular_data.values.mean()
v1 = pmv.Volume(regular_data, "test1", clims = np.array([0, total_std, 2 * total_std]))

In [21]:
s = pmv.Script([v1])
s.write("mtz_test2.py")

In [22]:
step_size = np.array([0.5,0.5,0.5])
m = gemmi.Mat33()
m.fromlist([[step_size[0],0.,0.],[0.,step_size[1],0.],[0.,0.,step_size[2]]])
transform = gemmi.Transform(m, gemmi.Vec3(*min_pos))
values = np.zeros(np.ceil((max_pos - min_pos) / step_size).astype(int), dtype = np.float32)
map.interpolate_values(values, transform)

In [23]:
positions = (np.array(m.tolist()).reshape(3,3) @ np.indices(values.shape).reshape(3,-1)).T + np.array(min_pos)

In [24]:
sorted_indices = np.lexsort((positions[:, 2], positions[:, 1], positions[:, 0]))

In [25]:
np.all(sorted_indices == np.arange(len(positions)))

True

In [26]:
regular_data = pmv.RegularData(positions, values.flatten(), step_count= np.array(values.shape))

In [27]:
np.array(values.shape)

array([193, 148, 186])

In [28]:
std = regular_data.values.std()
mean = regular_data.values.mean()

In [29]:
v1 = pmv.Volume(regular_data, "test1", clims = [mean, mean + 1 * std, mean + 2 * std])
v3 = pmv.IsoVolume(regular_data, "testr3",  clims = [mean + 0.5 * std, mean + 1 * std ,mean + 2 * std], alphas=[1, 1, 1], selection="sele", carve = 3, margin = 0.5)

In [30]:
s = pmv.Script([v1, v3])
s.write("mtz_test.py")

In [31]:
p1 = pmv.Points(positions[values.flatten() > total_std])
p1.to_script().write("mtz_points.py")